In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import os
import matplotlib.pyplot as plt
from numpy import int8, int16

src = '../finalCCRSdata/'


In [3]:
file_list = os.listdir(src)

# get a list of prefixes
# prefix_list = list(set([file.split('_')[0] for file in file_list if '_' in file and file.endswith('.csv')]))
prefix_list = ['Strains']


# make a dictionary of dataframes
for prefix in prefix_list:
    prefix_files = [file for file in file_list if file.startswith(
        prefix) and file.endswith('.csv')]
    df_list = [pd.read_csv(os.path.join(src, file)) for file in prefix_files]
    df_concat = pd.concat(
        df_list, keys=[(prefix, file) for file in prefix_files], ignore_index=True)


In [4]:
df_concat.columns


Index(['StrainId', 'StrainType', 'Name', 'IsDeleted', 'CreatedBy',
       'CreatedDate', 'UpdatedBy', 'UpdatedDate'],
      dtype='object')

In [5]:
df_concat = df_concat[['StrainId', 'StrainType', 'Name', 'IsDeleted', 
                       #'CreatedBy', 
                       'CreatedDate'
                       #'UpdatedBy', 'updatedDate'
                       ]]
#del df_list
#del file_list
#del df_concat


In [6]:
df_concat


,StrainId,StrainType,Name,IsDeleted,CreatedDate
0,401366,Hybrid,#Hashtag Hybrid,False,2020-06-26 00:00:00
1,20765,Hybrid,||||||| Glue EcoBudz,False,2018-08-02 00:00:00
2,172987,Hybrid,1:1 CBD,False,2020-09-29 00:00:00
3,172988,Hybrid,3 Bears OG,False,2020-08-20 00:00:00
4,172989,Hybrid,4 Juices,False,2020-02-13 00:00:00
...,...,...,...,...,...
490954,431585,Hybrid,ZZZZZZZZ,False,2021-12-05 08:54:00
490955,444849,Hybrid,ZZZZZZZZZZZZZ,False,2022-06-12 05:55:00
490956,271935,Hybrid,zzzzzzzzzzzzzzzzzzzzzz,False,2021-12-05 08:52:00
490957,271936,Hybrid,zzzzzzzzzzzzzzzzzzzzzzzz,False,2021-12-05 08:52:00


In [7]:
df_concat.isna().sum()


StrainId       0
StrainType     0
Name           4
IsDeleted      0
CreatedDate    0
dtype: int64

In [11]:
df_concat['Name'] = df_concat['Name'].fillna('Unknown')


In [12]:
df_concat.loc[df_concat.duplicated()]

# not sure what to do for the duplicates in this table, it appears that there is no primary key in this table. may need to modify the relationships in postgres


,StrainId,StrainType,Name,IsDeleted,CreatedDate


In [13]:
df_concat.dtypes


StrainId        int64
StrainType     object
Name           object
IsDeleted        bool
CreatedDate    object
dtype: object

In [14]:


df_concat = df_concat.astype({
    'StrainId' :        int,
    'StrainType' :     str,
    'Name' :           str,
}, errors='ignore')
df_concat['CreatedDate'] = pd.to_datetime(df_concat['CreatedDate'])

In [15]:
df_concat.dtypes

StrainId                int64
StrainType             object
Name                   object
IsDeleted                bool
CreatedDate    datetime64[ns]
dtype: object

In [16]:
df_concat.to_parquet('strains.parquet', index=False)
